# <h2 style= "text-align:center"> <b>Đồ án thực hành </b> </h2>
# <h2 style='text-align:center;font-size:32px'><b>Đề tài</b>: Natural Language Processing with Disaster Tweets</h1>
# <h2 style='text-align:center;font-size:32px'><b>Task 5-6-7</b>: Tạo đặc trưng - Mô hình hóa - Đánh giá </h1>

<hr/>
<div style="  color:#e3395e; padding: 15px; border-radius:5px;text-align: center; font-size: 40px">Tiếp cận theo hướng hiện đại</div>
<div style="  color:#e3395e; padding: 15px; border-radius:5px;text-align: center; font-size: 30px">Nội dung</div>
    
[1. Đọc dữ liệu](#1)
        
[2. Model](#2)
    
+    [2.1 Mô hình BERT phân loại chuỗi](#2_1)

        -    [2.1.1 Transform data](#2_1_1)

        -    [2.1.2 Train model](#2_1_2)
    
        -    [2.1.3 Đánh giá mô hình](#2_1_3)

+    [2.2 Mô hình RoBERTa](#2_2)
    
        -    [2.2.1 Transform data](#2_2_1)

        -    [2.2.2 Train model](#2_2_2)
    
        -    [2.2.3 Đánh giá mô hình](#2_2_3)
        
[3. Test](#3)

+    [3.1 Mô hình BERT phân loại chuỗi](#3_1)

+    [3.2 Mô hình RoBERTa](#3_2)

    

<a id="1"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">1. Đọc dữ liệu </div>

In [1]:
!pip install transformers

import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import re
from transformers import AutoTokenizer ,BertForSequenceClassification, AdamW, RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00


In [2]:
# # read train data
# train_df=pd.read_csv("data/new_train.csv")
# # read test data
# test_df=pd.read_csv("data/new_test.csv")
# read train data
train_df=pd.read_csv("train.csv").fillna('')
train_df['text'] = train_df['text'] +' '+ train_df['keyword'] +' '+ train_df['location']
train_df = train_df[['text','target']]
# read test data
test_df=pd.read_csv("test.csv").fillna('')
test_df['text'] = test_df['text'] + test_df['keyword'] + test_df['location']
test_df = test_df[['id','text']]

In [3]:
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
#Split data
train_df, valid_df = train_test_split(train_df, test_size = 0.15)

<a id="2"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
2. Model
</div>



<a id="2_1"></a>
<div style="  color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">2.1 Mô hình BERT phân loại chuỗi </div>



<a id="2_1_1"></a>
<div style="color:#e3395e; padding: 15px; border-radius:5px;  font-size: 25px">2.1.1 Transform data </div>

In [6]:
# Convert data to list
train_data=train_df.values.tolist()
val_data=valid_df.values.tolist()
test_data=test_df.values.tolist()

In [7]:
class Mydata(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

In [8]:
train=Mydata(train_data)
valid=Mydata(val_data)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [10]:
print("Model max_length: ", tokenizer.model_max_length)

Model max_length:  512


In [11]:
def collate_fn(data: list[tuple[str, int]]):
    texts = []
    labels = []
    for content, label in data:
        texts.append(content)
        labels.append(label)
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    labels = torch.tensor(labels)
    return input_ids, labels

In [12]:
train_dataloader = torch.utils.data.DataLoader(dataset = train, batch_size = 32, collate_fn = collate_fn, shuffle = True)
valid_dataloader = torch.utils.data.DataLoader(dataset = valid, batch_size = 32, collate_fn = collate_fn)

In [13]:
for input_ids, labels in train_dataloader:
    print(input_ids, labels)
    break

tensor([[  101,  3582,  1030,  ...,     0,     0,     0],
        [  101, 10731, 11113,  ...,     0,     0,     0],
        [  101,  2097,  2045,  ...,     0,     0,     0],
        ...,
        [  101,  1001, 23566,  ...,     0,     0,     0],
        [  101,  3099,  4473,  ...,     0,     0,     0],
        [  101,  2002,  1005,  ...,     0,     0,     0]]) tensor([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 1, 0, 1, 0])


In [14]:
torch.cuda.is_available()

True

<a id="2_1_2"></a>
<div style="color:#e3395e; padding: 15px; border-radius:5px;  font-size: 25px">2.1.2 Train model </div>

In [15]:
class MyBertForClassification(torch.nn.Module): # Class này kế thừa lớp Module của torch(Lớp này là lớp thường dùng để xây dưng mô hình của torch)
    def __init__(self, num_labels) -> None:
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels) # tải mô hình được train sẵn từ transformer
    def forward(self, input_ids, labels): # hàm này để tính các bước forward như các mô hình khác
        outputs = self.bert(input_ids=input_ids, labels=labels) # tính output của mô hình đã tải từ input là dữ liệu đầu vào và nhãn
        probs = torch.softmax(outputs['logits'], dim = -1) # hàm kích hoạt softmax để tính xác suất cho 2 nhãn đầu vào
        preds = torch.argmax(probs, dim = -1) # lẫy nhãn có xác suất cao nhất
        outputs['preds'] = preds
        return outputs

In [16]:
model = MyBertForClassification(2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [17]:
# Chuyển mô hình lên GPU
model.cuda()

MyBertForClassification(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_featu

In [18]:
LR = 4e-6
EPOCH = 4
LOG_STEP = 30

In [19]:
optimizer = AdamW(model.parameters(), lr=LR) # hàm tối ưu hoá

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
print(f"Train Loader Step: {len(train_dataloader)}")
print(f"Valid Loader Step: {len(valid_dataloader)}")

Train Loader Step: 203
Valid Loader Step: 36


In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = []
precision = []
recall = []
f1 = []
loss_list = []

for epoch in range(EPOCH):
    running_loss = 0.0
    print(f"Epoch {epoch + 1}:")
    model.train()  # Chuyển sang chế độ huấn luyện
    for i, (input_id, label) in enumerate(train_dataloader):
        input_ids = input_id
        labels = label

        input_ids = input_ids.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(input_ids, labels) # tính toán output từ dữ liệu ở trên
        loss = outputs['loss'] # tính độ lỗi của
        loss.backward() # thực hiện backward để điều chỉnh tham số
        optimizer.step() # tối ưu hoá tham số

        running_loss += loss.item() # cộng độ lỗi đề đánh giá

        if i % LOG_STEP == LOG_STEP-1:
            print('[Epoch %d, Batch %d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / LOG_STEP))
            running_loss = 0.0

    model.eval()
    valid_loss = 0
    true_labels = []
    pred_labels = []

    with torch.no_grad(): # tât chức năng tính toán gradient vì phần đánh giá không có điều chỉnh tham số
        for step, (input_ids, labels) in enumerate(valid_dataloader):
            input_ids = input_ids.cuda()
            labels = labels.cuda()
            outputs = model(input_ids, labels)
            loss = outputs['loss']
            valid_loss += loss.item()


            true_labels.extend(labels.tolist())
            pred_labels.extend(outputs['preds'].tolist())

    accuracy.append(accuracy_score(true_labels, pred_labels))
    precision.append(precision_score(true_labels, pred_labels, average='macro'))
    recall.append(recall_score(true_labels, pred_labels, average='macro'))
    f1.append(f1_score(true_labels, pred_labels, average='macro'))
    loss_list.append((valid_loss / len(valid_dataloader)))
    print(f"Valid loss: {valid_loss / len(valid_dataloader)}")

# Lưu mô hình
torch.save(model.state_dict(), 'bert_classifier.pth')
# model.load_state_dict(torch.load('bert_classifier.pth'))
# model.eval()

Epoch 1:
[Epoch 1, Batch 30] loss: 0.674
[Epoch 1, Batch 60] loss: 0.635
[Epoch 1, Batch 90] loss: 0.561
[Epoch 1, Batch 120] loss: 0.538
[Epoch 1, Batch 150] loss: 0.484
[Epoch 1, Batch 180] loss: 0.465
Valid loss: 0.4546455633309152
Epoch 2:
[Epoch 2, Batch 30] loss: 0.414
[Epoch 2, Batch 60] loss: 0.428
[Epoch 2, Batch 90] loss: 0.420
[Epoch 2, Batch 120] loss: 0.413
[Epoch 2, Batch 150] loss: 0.392
[Epoch 2, Batch 180] loss: 0.398
Valid loss: 0.4045538115832541
Epoch 3:
[Epoch 3, Batch 30] loss: 0.362
[Epoch 3, Batch 60] loss: 0.335
[Epoch 3, Batch 90] loss: 0.379
[Epoch 3, Batch 120] loss: 0.387
[Epoch 3, Batch 150] loss: 0.338
[Epoch 3, Batch 180] loss: 0.382
Valid loss: 0.3955279737710953
Epoch 4:
[Epoch 4, Batch 30] loss: 0.309
[Epoch 4, Batch 60] loss: 0.356
[Epoch 4, Batch 90] loss: 0.329
[Epoch 4, Batch 120] loss: 0.375
[Epoch 4, Batch 150] loss: 0.307
[Epoch 4, Batch 180] loss: 0.350
Valid loss: 0.42175019159913063


<a id="2_1_3"></a>
<div style="color:#e3395e; padding: 15px; border-radius:5px;  font-size: 25px">2.1.3 Đánh giá mô hình </div>

In [22]:
df_score = pd.DataFrame({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Loss': loss_list }, index=
            ['Epoch 1','Epoch 2','Epoch 3', 'Epoch 4'])

In [23]:
df_score

,Accuracy,Precision,Recall,F1,Loss
Epoch 1,0.784588,0.784383,0.788471,0.783776,0.454646
Epoch 2,0.828371,0.830748,0.820190,0.823485,0.404554
Epoch 3,0.832750,0.835807,0.824314,0.827832,0.395528
Epoch 4,0.823993,0.821637,0.820932,0.821268,0.421750


Hiệu suất của mô hình được cải thiện qua mỗi epoch trong đó `Accuracy`,	`Precision`, `Recall`, `F1` tăng dần và `Loss` giảm dần

<a id="2_2"></a>
<div style=" color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">
2.2 Mô hình ROBERTA phân loại chuỗi</div>



<a id="2_2_1"></a>
<div style="color:#e3395e; padding: 15px; border-radius:5px;  font-size: 25px">2.2.1 Transform data </div>

In [25]:
#Convert data to numpy array
X_train = train_df['text'].to_numpy()
y_train = train_df['target'].to_numpy()

X_val = valid_df['text'].to_numpy()
y_val = valid_df['target'].to_numpy()

X_test = test_df['text'].to_numpy()

### Tokenizer
Tham khảo: [Disaster tweets: BERTs+Ensembling](https://www.kaggle.com/code/andreynesterov/disaster-tweets-berts-ensembling#Data-Preprocessing-)

In [26]:
# Load tokenizer model
tokenizer_roberta = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)

In [27]:
def get_tokenizer_max_len(data):
    '''
    Trả về số lượng token lớn nhất khi mã hóa tất cả các câu có trong `data`
    '''
    token_lens = []
    for txt in data:
        tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
        token_lens.append(len(tokens))
    max_length=np.max(token_lens)
    return max_length

In [28]:
# tìm số lượng token lớn nhất mà 1 câu có thể tạo ra
max_train = get_tokenizer_max_len(X_train)
max_val = get_tokenizer_max_len(X_val)

MAX_LEN = max_train if max_train > max_val else max_val
MAX_LEN

91

In [29]:
def tokenizer_fun(data, max_len=MAX_LEN) :
    '''
    mã hóa các câu thành token với độ dài bằng nhau (max_len), cho phép các token đặc biệt và thêm phần đệm.
    Kết quả trả về 2 array: mảng token và mảng cho biết vị trí tương ứng trong mảng token có là giá trị quan trọng hay không
    '''
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)

In [30]:
def get_tokenized_ds(X_data, y_data, tokenizer, batch_size=32, prefetch=True):
    X_data_tokenized = tokenizer(X_data)
    X_data_tokenized_ds = tf.data.Dataset.from_tensor_slices(X_data_tokenized)

    # nếu dữ liệu đầu vào là tập train, val
    if len(y_data) > 0:
        y_data_ds = tf.data.Dataset.from_tensor_slices(y_data)
        data_ds = tf.data.Dataset.zip((X_data_tokenized_ds, y_data_ds))
    else:   # dữ liệu đầu vào là tập test
        data_ds = tf.data.Dataset.from_tensor_slices((X_data_tokenized,))

    # chia dữ liệu thành các batch
    if batch_size>0:
        data_ds = data_ds.batch(batch_size)

    # tự động chuẩn bị dữ liệu cho batch tiếp theo
    if prefetch:
        data_ds = data_ds.prefetch(tf.data.AUTOTUNE)
    return data_ds

In [31]:
train_roberta_ds = get_tokenized_ds(X_train, y_train, tokenizer_fun)
val_roberta_ds = get_tokenized_ds(X_val, y_val, tokenizer_fun)
test_roberta_ds = get_tokenized_ds(X_test, [], tokenizer_fun, prefetch=False)
train_roberta_ds.element_spec, val_roberta_ds.element_spec, test_roberta_ds.element_spec

(((TensorSpec(shape=(None, 91), dtype=tf.int64, name=None),
   TensorSpec(shape=(None, 91), dtype=tf.int64, name=None)),
  TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
 ((TensorSpec(shape=(None, 91), dtype=tf.int64, name=None),
   TensorSpec(shape=(None, 91), dtype=tf.int64, name=None)),
  TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
 ((TensorSpec(shape=(None, 91), dtype=tf.int64, name=None),
   TensorSpec(shape=(None, 91), dtype=tf.int64, name=None)),))

<a id="2_2_2"></a>
<div style="color:#e3395e; padding: 15px; border-radius:5px;  font-size: 25px">2.2.2 Train model </div>

Tham khảo: [Disaster tweets: BERTs+Ensembling](https://www.kaggle.com/code/andreynesterov/disaster-tweets-berts-ensembling#Data-Preprocessing-)


In [32]:
model_roberta = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [33]:
%%time
# Callback function: nếu val_accuracy không cải thiện trong 2 epochs liên tiếp, lr sẽ được giảm đi một hệ số 0.3.
reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(patience=2, monitor="val_accuracy", factor=0.3, verbose=1)

# Callback function: nếu val_accuracy không cải thiện trong 3 epochs liên tiếp
# Quá trình huấn luyện sẽ được dừng và các trọng số tốt nhất sẽ được khôi phục.
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)

# tối ưu hóa với Adam optimizer
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

# biên dịch và huấn luyện mô hình
model_roberta.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer=opt, metrics=["accuracy"])
model_roberta.fit(
        train_roberta_ds,
        validation_data=val_roberta_ds,
        epochs=10,
        callbacks=[early_stopping_cb, reduce_lr_cb])

Epoch 1/10
203/203 [==============================] - 188s 668ms/step - loss: 0.4807 - accuracy: 0.7704 - val_loss: 0.3784 - val_accuracy: 0.8380 - lr: 1.0000e-05
Epoch 2/10
203/203 [==============================] - 126s 621ms/step - loss: 0.3570 - accuracy: 0.8523 - val_loss: 0.3724 - val_accuracy: 0.8415 - lr: 1.0000e-05
Epoch 3/10
203/203 [==============================] - 122s 603ms/step - loss: 0.3031 - accuracy: 0.8809 - val_loss: 0.3952 - val_accuracy: 0.8354 - lr: 1.0000e-05
Epoch 4/10
203/203 [==============================] - ETA: 0s - loss: 0.2589 - accuracy: 0.8982
Epoch 4: ReduceLROnPlateau reducing learning rate to 2.9999999242136253e-06.
203/203 [==============================] - 122s 603ms/step - loss: 0.2589 - accuracy: 0.8982 - val_loss: 0.4422 - val_accuracy: 0.8292 - lr: 1.0000e-05
Epoch 5/10
203/203 [==============================] - 123s 604ms/step - loss: 0.1916 - accuracy: 0.9241 - val_loss: 0.4759 - val_accuracy: 0.8205 - lr: 3.0000e-06
CPU times: user 7min 19

<a id="2_2_3"></a>
<div style="color:#e3395e; padding: 15px; border-radius:5px;  font-size: 25px">2.2.3 Đánh giá mô hình </div>

### Dự đoán trên tập validation

In [34]:
y_val_predictions = model_roberta.predict(val_roberta_ds)

36/36 [==============================] - 21s 191ms/step


In [35]:
# Độ chính xác và độ lỗi của mô hình
pd.Series(np.argmax(y_val_predictions.logits, axis=1) == y_val).value_counts(normalize=True)

True     0.841506
False    0.158494
dtype: float64

Mô hình có độ chính xác 84,676% trên tập validation.
> So sánh kết quả khi chạy trên tập validation ta thấy mô hình RoBERTa hoạt động tốt hơn mô hình BERT.

<a id="3"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
3. Test
</div>

<a id="3_1"></a>
<div style="  color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">3.1 Mô hình BERT phân loại chuỗi </div>



In [36]:
def collate_fn(data: list[tuple[str, int]]):
    texts = []
    ids = []
    for id, content in data:
        texts.append(content)
        ids.append(id)
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    ids = torch.tensor(ids)
    return input_ids, ids

In [37]:
inputs,ids=collate_fn(test_data)

In [38]:
test=Mydata(test_data)

In [39]:
test_dataloader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 1, collate_fn = collate_fn)

In [40]:
result_test={
    'id':[],
    'target':[]
}

In [41]:
with torch.no_grad():
    for step, (input_ids, idx) in enumerate(test_dataloader):
        input_ids = input_ids.cuda()
        # labels = labels.cuda()
        outputs = model(input_ids=input_ids, labels=None)
        preds = outputs['preds']
        span = preds[0].item()
        result_test['id'].append(idx.item())
        result_test['target'].append(span)

In [42]:
df=pd.DataFrame(result_test)

In [43]:
df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [45]:
df.to_csv("submission/result1.csv",index=False)

### Kết quả 📜📜




![result](image/result1.png)  

<a id="3_2"></a>
<div style=" color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">
3.2 Mô hình ROBERTA phân loại chuỗi</div>



In [46]:
# dự đoán cho tập test
y_predictions = model_roberta.predict(test_roberta_ds)

102/102 [==============================] - 24s 198ms/step


In [47]:
result = pd.DataFrame()
result['id'] = test_df[['id']].copy()

In [48]:
# Chọn cột có giá trị lớn hơn ở mỗi dòng
new_arr = np.argmax(y_predictions.logits, axis=1)
result['target'] = new_arr

In [49]:
result.to_csv("submission/result2.csv",index=False)

### Kết quả 📜📜




![result](image/result2.png)  

<hr/>
<div style="  color:#e3395e; padding: 15px; border-radius:5px;text-align: center; font-size: 40px">Tiếp cận theo hướng truyền thống</div>
<div style="  color:#e3395e; padding: 15px; border-radius:5px;text-align: center; font-size: 30px">Nội dung</div>
    
[1. Đọc dữ liệu](#_1)
        
[2. Quá trình tạo đăc trưng](#_2)

[3. Thử nghiệm](#_3)

[4. Kiểm tra kết quả cuối cùng](#_4)


<a id="a"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">1. Đọc dữ liệu </div>

### Các thư viện cơ bản

In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [51]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

### Đọc lại các file đã qua xử lí

In [53]:
data_train = pd.read_csv('data/new_train.csv').fillna('None')
data_test = pd.read_csv('data/new_test.csv').fillna('None')

In [54]:
data_train

,id,text,extras,target
0,1,Our Deeds are the Reason of this May ALLAH Fo...,#earthquake,1
1,4,Forest fire near La Ronge Sask. Canada,,1
2,5,All residents asked to 'shelter in place' are ...,,1
3,6,"13,000 people receive evacuation orders in Ca...",#wildfires,1
4,7,Just got sent this photo from Ruby as smoke f...,#Alaska #wildfires,1
...,...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,http://t.co/STfMbbZFB5,1
7609,10870,The out of control wild fires in California ...,@aria_ahrary @TheTawniest,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii.,http://t.co/zDtoyd8EbJ,1
7611,10872,Police investigating after an e-bike collided ...,,1


In [55]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [56]:
nltk.download('stopwords')
english_stopwords=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

<a id="_2"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">2. Quá trình tạo đặc trưng </div>

In [58]:
#vectorization
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import FeatureUnion
TVectorizer = TfidfVectorizer(ngram_range = (1,2),min_df=10)
CVectorizer = CountVectorizer(min_df=5)

#ở đây, các mục thuần text sẽ được TfidfVectorizer
train_text = TVectorizer.fit_transform(data_train['text'])
#còn các phần tử thuộc extra sẽ là CountVectorizer, bởi do số lượng xuất hiện ít và có thể là đặc trưng chính
train_extras = CVectorizer.fit_transform(data_train['extras'])

test_text = TVectorizer.transform(data_test['text'])
test_extras = CVectorizer.transform(data_test['extras'])

#Nối lại 2 dataframe
X_train_text = pd.DataFrame(train_text.toarray(), columns=TVectorizer.get_feature_names_out()).join(
    pd.DataFrame(train_extras.toarray(), columns=CVectorizer.get_feature_names_out()),rsuffix='_extras')
X_test_text =  pd.DataFrame(test_text.toarray(), columns=TVectorizer.get_feature_names_out()).join(
    pd.DataFrame(test_extras.toarray(), columns=CVectorizer.get_feature_names_out()),rsuffix='_extras')

y_train_target=data_train['target']

# we're NOT using .fit_transform() here. Using just .transform() makes sure that the tokens in the train vectors are the only ones mapped to the test vectors -
#i.e. that the train and test vectors use the same set of tokens.

<a id="_3"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">3. Thử nghiệm </div>

In [59]:
X_train,X_val,y_train,y_val = train_test_split(X_train_text,y_train_target,test_size=0.2,random_state=200)

#### Logistic Regression

In [60]:
lr_model=LogisticRegression(solver ='liblinear' ,random_state=100,penalty='l2')
lr_model.fit(X_train,y_train)

LogisticRegression(random_state=100, solver='liblinear')

In [61]:
from sklearn.metrics import accuracy_score

y_pred=lr_model.predict(X_val)
val_accuracy=accuracy_score(y_val,y_pred)
print('Accuracy : ',round(val_accuracy *100,2),'%')

Accuracy :  80.37 %


#### SVM (Support vector machine)

In [62]:
from sklearn.svm import SVC
svc_model=SVC()
svc_model.fit(X_train,y_train)

SVC()

In [63]:
from sklearn.metrics import accuracy_score

y_pred=svc_model.predict(X_val)
val_accuracy=accuracy_score(y_val,y_pred)
print('Accuracy : ',round(val_accuracy *100,2),'%')

Accuracy :  80.43 %


#### RandomForest

In [64]:
from sklearn.ensemble import RandomForestClassifier
rd_model = RandomForestClassifier(n_estimators=140)
rd_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=140)

In [65]:
from sklearn.metrics import accuracy_score

y_pred=rd_model.predict(X_val)
val_accuracy=accuracy_score(y_val,y_pred)
print('Accuracy : ',round(val_accuracy *100,2),'%')

Accuracy :  78.73 %


=> LogisticRegression cho kết quả nhanh và độ chính xác khá hiệu quả

Tập trung tối ưu LogisticRegression

Ở trên ta chia tập train thành 2 tập để có thể tạo mô hình bằng 1 tập sau đó dùng tập còn lại để kiểm tra

Ở phần này làm luôn cả bộ, không tách thành 2 tập nữa vì sau khi tối ưu hơn cái mô hình, lấy luôn mô hình đó để dự đoán file test

In [66]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
#thử kiểm tra xem với tất cả feature đang có thì cv được bao nhiêu?
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=200)
cv_score = cross_val_score(lr_model, X_train_text, y_train_target, cv=cv, scoring='f1')
cv_score.mean()

0.7429935382372316

In [67]:
# Dòng này dùng để tìm ra các từ (feature) có ảnh hưởng lớn nhất
_coef_ = pd.DataFrame.from_dict(
    dict(zip(('coef','feature'),
             zip(*sorted(tuple(zip(lr_model.coef_[0],X_train_text.columns)),key = lambda x:x[0],reverse=True))
        )))

In [68]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.add_trace(go.Bar(y=_coef_['coef'][:15], x=_coef_['feature'][:15],name='taget==1'),1, 1)
fig.add_trace(go.Bar(y=_coef_['coef'][-15:-1], x=_coef_['feature'][-15:-1],name='taget==0'),1, 2)
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=True,title={'text': "Top feature có ảnh hưởng"})
fig.show()

In [69]:
from sklearn.feature_selection import RFECV
#thư viện RFECV để trích feature
steps = 20
n_features = len(X_train_text.columns)
X_range = [1] + list(range(n_features,1,-20)[::-1])
#f1 sẽ là chỉ số đánh giá
rfecv = RFECV(estimator=lr_model, step=steps, cv=cv, scoring='f1')

rfecv.fit(X_train_text,y_train_target)

RFECV(cv=ShuffleSplit(n_splits=5, random_state=200, test_size=0.2, train_size=None),
      estimator=LogisticRegression(random_state=100, solver='liblinear'),
      scoring='f1', step=20)

In [70]:
import plotly.graph_objects as go
import numpy as np

x = np.arange(10)

fig = go.Figure()
fig.add_trace(go.Scatter(x=X_range, y=rfecv.cv_results_['mean_test_score'],
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=[sum(rfecv.ranking_==1),sum(rfecv.ranking_==1)], y=[0,1],
                    mode='lines',
                    name='max'))
fig.update_layout(title = {'text' : 'Số lượng feature được chọn và f1 trung bình của nó'},showlegend=True)
fig.show()

#### Chọn feature

In [71]:
Recursive_Feature_Elimination = X_train_text[X_train_text.columns[rfecv.ranking_==1]]
X_test_text_Recursive_Feature_Elimination = X_test_text[X_test_text.columns[rfecv.ranking_==1]]

In [72]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=200)
cv_score = cross_val_score(lr_model, Recursive_Feature_Elimination, y_train_target, cv=cv, scoring='f1')
cv_score.mean()


0.7523777620471188

In [73]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.add_trace(go.Bar(y=_coef_['coef'][:15], x=_coef_['feature'][:15],name='taget==1'),1, 1)
fig.add_trace(go.Bar(y=_coef_['coef'][-15:-1], x=_coef_['feature'][-15:-1],name='taget==0'),1, 2)
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=True,title={'text': "Top feature có ảnh hưởng sau khi chọn lại feature"})
fig.show()

<a id="_4"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">4. Kiểm tra kết quả cuối cùng</div>

In [74]:
lr_model.fit(Recursive_Feature_Elimination,y_train_target)
y_pred=lr_model.predict(X_test_text_Recursive_Feature_Elimination)

In [75]:
data_test['target'] = y_pred

In [76]:
data_test[['id','target']].to_csv('submission/result3.csv',index=False)

<div style="color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
Nhận xét
</div>